In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import sys 

In [ ]:
# this cell is specifically for PoolNet (ground truth images)

gt_list = []
gt_paths = []
path = './data/SOD/Imgs'  # ground image path directory
test = os.listdir(path)
for i in test: 
    if(i[-3:] == 'png'):
        gt_list.append(i)
gt_list = sorted(gt_list)
for i in range(len(gt_list)):
    gt_paths.append(os.path.join(path, gt_list[i]))

gt_paths = sorted(gt_paths)

In [ ]:
# this cell is specifically for PoolNet (predicted images)

pred_paths = []
path = './results/run-1-sal-s' # predicted image directory
test_pred = sorted(os.listdir(path))
for i in range(len(test_pred)):
    pred_paths.append(os.path.join(path, test_pred[i]))

pred_paths = sorted(pred_paths)

In [ ]:
def prec_rec(y_true, y_pred, beta2):
    
    eps = sys.float_info.epsilon
    tp = torch.sum(y_true * y_pred)
    all_p_pred = torch.sum(y_pred)
    all_p_true = torch.sum(y_true)
    
    prec = (tp + eps) / (all_p_pred + eps)
    rec = (tp + eps) / (all_p_true + eps)
    # print(prec)
    # print(rec)
    
    return prec, rec

In [ ]:
overall_mae = 0
for j in range(len(gt_paths)):

        gt = np.array(Image.open(gt_paths[j]).convert('LA')) / 255
        pred = np.array(Image.open(pred_paths[j]).convert('LA')) / 255 
        mae = np.sum(np.abs(pred - gt)) / (pred.shape[:2][0] * pred.shape[:2][1])
        
        gt_arr = torch.from_numpy(np.array(gt)).float()
        pred_arr = torch.from_numpy(np.array(pred)).float()
        threshold = 216
        y_pred = torch.ge(pred_arr, threshold).float()
        y_true = torch.ge(gt_arr, 128).float()
        y_true1 = torch.reshape(y_true, (1,-1))
        y_pred1 = torch.reshape(y_pred, (1,-1))
        
        prec, rec = prec_rec(y_true1, y_pred1,0.3)

        total_prec = total_prec + prec
        total_rec = total_rec + rec

        overall_mae = overall_mae + mae

        print(j)

beta2 = 0.3 
overall_fb = (1+beta2) * (total_prec * total_rec) / ((beta2 * total_prec + total_rec) * len(gt_paths))
print('overall_mae', overall_mae )
print('overall_fb', overall_fb )